In [1]:
import sys

sys.path.insert(0, "..")
sys.path.insert(0, "../disc_cop")
from disc_cop.envs import ENV_ID_TO_NAME

import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import torch
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
result_dir = "/Volumes/Expansion/icml_2025/avg_ope/cc_results/results/saved_models"
stats_file = "/Volumes/Expansion/icml_2025/avg_ope/cc_results/results/processed_best.feather"

In [3]:
stats = []

for env_name in ENV_ID_TO_NAME.values():
    if not os.path.isdir(os.path.join(result_dir, env_name)):
        print("{} not processed".format(env_name))
        continue

    for run_file in tqdm(os.listdir(os.path.join(result_dir, env_name)), desc=env_name):
        if run_file.startswith("."):
            continue

        prefix = run_file.split("-")[:4]

        for seed in range(10):
            torch_path = os.path.join(result_dir, env_name, run_file, "seed_{}-curr_best.pt".format(seed))
            data = torch.load(torch_path)
            curr_loss = data["loss"]
            curr_steps = data["steps"]

            stats.append(
                dict(
                    variant=run_file,
                    env_name=env_name,
                    seed=seed,
                    **{
                        "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix
                    },
                    loss=curr_loss,
                    steps=curr_steps,
                )
            )

ant not processed
halfcheetah not processed
hopper not processed
swimmer not processed
walker not processed
acrobot not processed
cartpole not processed
mountain_car not processed


In [4]:
stats = pd.DataFrame(stats)
stats.to_feather(stats_file)

OSError: Cannot save file into a non-existent directory: '/Volumes/Expansion/icml_2025/avg_ope/cc_results/results'

In [ ]:
stats

In [5]:
stats = pd.read_feather("/Users/chanb/research/ualberta/Avg_OPE/processed_best.feather")

In [6]:
stats

,variant,env_name,seed,random_weight,discount_factor,max_ep,max_len,loss,steps
0,random_weight_1.4-discount_factor_0.9-max_ep_2...,ant,0,1.4,0.9,20,200,3.592368e-12,57175
1,random_weight_1.4-discount_factor_0.9-max_ep_2...,ant,1,1.4,0.9,20,200,2.012213e-09,149105
2,random_weight_1.4-discount_factor_0.9-max_ep_2...,ant,2,1.4,0.9,20,200,8.447719e-10,63955
3,random_weight_1.4-discount_factor_0.9-max_ep_2...,ant,3,1.4,0.9,20,200,9.112059e-11,102410
4,random_weight_1.4-discount_factor_0.9-max_ep_2...,ant,4,1.4,0.9,20,200,7.418625e-11,233055
...,...,...,...,...,...,...,...,...,...
27995,random_weight_0.2-discount_factor_0.8-max_ep_5...,mountain_car,5,0.2,0.8,50,80,7.084168e-05,10
27996,random_weight_0.2-discount_factor_0.8-max_ep_5...,mountain_car,6,0.2,0.8,50,80,1.528097e-04,0
27997,random_weight_0.2-discount_factor_0.8-max_ep_5...,mountain_car,7,0.2,0.8,50,80,3.751824e-05,0
27998,random_weight_0.2-discount_factor_0.8-max_ep_5...,mountain_car,8,0.2,0.8,50,80,1.054977e-04,0


In [23]:
stats[
    (stats["env_name"] == "halfcheetah")
    & (stats["max_len"] == "50")
    & (stats["max_ep"] == "160")
    & (stats["discount_factor"] == "0.95")
    & (stats["seed"] == 0)
]

,variant,env_name,seed,random_weight,discount_factor,max_ep,max_len,loss,steps
4770,random_weight_1.8-discount_factor_0.95-max_ep_...,halfcheetah,0,1.8,0.95,160,50,4.735872e-11,57795
5310,random_weight_2.8-discount_factor_0.95-max_ep_...,halfcheetah,0,2.8,0.95,160,50,2.374834e-13,13130
5920,random_weight_2.0-discount_factor_0.95-max_ep_...,halfcheetah,0,2.0,0.95,160,50,1.263975e-13,66490
6240,random_weight_1.4-discount_factor_0.95-max_ep_...,halfcheetah,0,1.4,0.95,160,50,1.435944e-10,97165
6950,random_weight_2.4-discount_factor_0.95-max_ep_...,halfcheetah,0,2.4,0.95,160,50,3.158456e-12,167960


In [24]:
stats[
    (stats["env_name"] == "halfcheetah")
    & (stats["max_len"] == "50")
    & (stats["max_ep"] == "160")
    & (stats["discount_factor"] == "0.95")
    & (stats["seed"] == 1)
]

,variant,env_name,seed,random_weight,discount_factor,max_ep,max_len,loss,steps
4771,random_weight_1.8-discount_factor_0.95-max_ep_...,halfcheetah,1,1.8,0.95,160,50,2.030844e-11,221060
5311,random_weight_2.8-discount_factor_0.95-max_ep_...,halfcheetah,1,2.8,0.95,160,50,8.044461e-15,36060
5921,random_weight_2.0-discount_factor_0.95-max_ep_...,halfcheetah,1,2.0,0.95,160,50,1.987551e-13,65805
6241,random_weight_1.4-discount_factor_0.95-max_ep_...,halfcheetah,1,1.4,0.95,160,50,1.142591e-10,96925
6951,random_weight_2.4-discount_factor_0.95-max_ep_...,halfcheetah,1,2.4,0.95,160,50,7.196688e-16,190310


In [25]:
stats[
    (stats["env_name"] == "halfcheetah")
    & (stats["max_len"] == "50")
    & (stats["max_ep"] == "160")
    & (stats["discount_factor"] == "0.95")
    & (stats["seed"] == 2)
]

,variant,env_name,seed,random_weight,discount_factor,max_ep,max_len,loss,steps
4772,random_weight_1.8-discount_factor_0.95-max_ep_...,halfcheetah,2,1.8,0.95,160,50,7.671019e-11,208215
5312,random_weight_2.8-discount_factor_0.95-max_ep_...,halfcheetah,2,2.8,0.95,160,50,6.668066e-13,120715
5922,random_weight_2.0-discount_factor_0.95-max_ep_...,halfcheetah,2,2.0,0.95,160,50,6.651390e-13,141270
6242,random_weight_1.4-discount_factor_0.95-max_ep_...,halfcheetah,2,1.4,0.95,160,50,8.305535e-12,234075
6952,random_weight_2.4-discount_factor_0.95-max_ep_...,halfcheetah,2,2.4,0.95,160,50,1.738376e-12,110960
